# Hello Radiomics example: using the feature extractor to calculate features

This example shows how to use the radiomics package and the feature extractor.
The feature extractor handles preprocessing, and then calls the needed featureclasses to calculate the features.
It is also possible to directly instantiate the feature classes. However, this is not recommended for use outside debugging or development. For more information, see `helloFeatureClass`.

In [ ]:
import os
import logging
from radiomics.featureextractor import RadiomicsFeaturesExtractor
import radiomics

## Setting up logging

In [ ]:
logger = logging.getLogger('radiomics')  # Parent logger for the radiomics package

# Enable writing out the log using radiomics logger
radiomics.debug()  # Switch on radiomics logging from level=DEBUG (default level=WARNING)
# Alternative; set logging to a specific level
# logger.setLevel(logging.INFO)

# Prevent radiomics logger from printing out log entries with level < WARNING to the console
logger.handlers[0].setLevel(logging.WARNING)  # This limits the log messages printed to stdErr to level warning or higher
logger.propagate = False  # Do not pass log messages on to root logger

# Write out all log entries to a file, overwrite previous log
handler = logging.FileHandler(filename='testLog.txt', mode='w')
formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # This determines how a log message should be formatted
handler.setFormatter(formatter)
logger.addHandler(handler)

## Getting the testcase

In [ ]:
testCase = 'brain1'
dataDir = os.path.dirname(os.path.abspath(__file__)) + os.path.sep + ".." + os.path.sep + "data"
imageName = str(dataDir + os.path.sep + testCase +'_image.nrrd')
maskName = str(dataDir + os.path.sep + testCase + '_label.nrrd')

if not os.path.exists(imageName):
  print 'Error: problem finding input image', imageName
if not os.path.exists(maskName):
  print 'Error: problem finding input labelmap', maskName

## Initializing the feature extractor

#### Extraction Settings

In [ ]:
kwargs = {}
kwargs['binWidth'] = 25
kwargs['resampledPixelSpacing'] = None
# kwargs['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
kwargs['interpolator'] = 'sitkBSpline'
kwargs['verbose'] = True

extractor = RadiomicsFeaturesExtractor(**kwargs)

#### Input images: applying filters

In [ ]:
# By default, only 'original' (no filter applied) is enabled. Optionally enable some filters:

# extractor.enableInputImageByName('wavelet')
# extractor.enableInputImageByName('log', customArgs={'sigma':[3.0]})
# extractor.enableInputImageByName('square')
# extractor.enableInputImageByName('squareroot')
# extractor.enableInputImageByName('exponential')
# extractor.enableInputImageByName('logarithm')

# Alternative; set filters in one operation 
# This updates current enabled input images, i.e. overwrites custom settings. 
# However, input images already enabled, but not passed in this call, are not disabled.

# extractor.enableInputImages(wavelet={}, log={'sigma':[3.0]})

print "Enabled input images:"
for imageType in extractor.inputImages.keys():
    print '\t' + imageType

#### Feature classes: setting which feature(classes) need to be calculated

In [ ]:
# Disable all classes
extractor.disableAllFeatures()

# Enable all features in firstorder
extractor.enableFeatureClassByName('firstorder')

# Alternative; only enable 'Mean' and 'Skewness' features in firstorder
# extractor.enableFeaturesByName(firstorder=['Mean', 'Skewness'])

## Getting the docstrings of the active features

In [ ]:
print "Active features:"
for cls, features in extractor.enabledFeatures.iteritems():
  if len(features) == 0:
    features = extractor.getFeatureNames(cls)
  for f in features:
    print f
    print eval('extractor.featureClasses["%s"].get%sFeatureValue.__doc__' % (cls, f))

## Calculating the values of the active features

In [ ]:
print "Calculating features"
featureVector = extractor.execute(imageName, maskName)

for featureName in featureVector.keys():
  print "Computed %s: %s" % (featureName, featureVector[featureName])